<a href="https://colab.research.google.com/github/tzhsu211/CVS/blob/main/Finetune0205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [2]:
import torch
import pandas as pd
import numpy as np
import json
import os
import warnings
import random
from accelerate import Accelerator
from transformers import BertTokenizerFast, EarlyStoppingCallback, AutoModelForSequenceClassification, Trainer, TrainingArguments
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datasets import Dataset
import optuna


In [3]:
torch.cuda.is_available()

True

In [4]:
warnings.filterwarnings('ignore')

### Load data
1. Load raw CSV file
2. Standardize x and y (drop unnecessary columns, then scale the rating)
3. Split train and test dataset
4. Load tokenizer
5. Load pre-trained model

In [5]:
df = pd.read_csv('./cvs_products.csv')
df1 = df.drop(['product', 'store', 'link', 'CVS'], axis = 1)
scaler = StandardScaler()
df1['rating_standard'] = scaler.fit_transform(df['rating'].values.reshape(-1, 1))
X_train, X_test, y_train, y_test = train_test_split(df1['review'], df1['rating_standard'], test_size=0.1, random_state=42)
train_df = pd.DataFrame({'text': X_train, 'label': y_train.astype(float)})
test_df = pd.DataFrame({'text': X_test, 'label': y_test.astype(float)})

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
def preprocess_function(examples, tokenizer):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

google_bert_tokenizer = BertTokenizerFast.from_pretrained('google-bert/bert-base-chinese')
google_bert_train_dataset = train_dataset.map(lambda x: preprocess_function(x, google_bert_tokenizer), batched=True)
google_bert_test_dataset = test_dataset.map(lambda x: preprocess_function(x, google_bert_tokenizer), batched=True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

Map:   0%|          | 0/8656 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

In [7]:
def compute_metrics(p):
    preds, labels = p
    preds = preds.squeeze()
    mse = mean_squared_error(labels, preds)
    r2 = r2_score(labels, preds)
    return {"mse": mse, "r2": r2}

In [9]:
checkpoint_path = './google_bert'
google_bert_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, config=f'{checkpoint_path}/config.json')


**Optuna** hyperparameter tuning

1. **Hyperparameter Selection for Fine-Tuning**:
   - **Learning Rate**: Chosen from a log-uniform distribution between `1e-5` and `1e-3`.
   - **Dropout Rate**: Set between `0.1` and `0.5`.
   - **Batch Size**: Two options are chosen, `32` and `64`.
   - **Weight Decay**: Optimized within the range of `0.01` to `0.1`.

   These hyperparameters directly affect the training process, and Optuna will explore different combinations to determine which yields the best performance.

2. **Comparing with Pre-Trained Model Config**:
   - **Warm-up Steps**: The default warm-up steps were set to `500` from the pre-trained model's training arguments. In this fine-tuning setup, they were reduced to `300` to speed up training and explore hyperparameter effects more efficiently.
   - **Number of Epochs**: Reduced to `3` to quickly test the random hyperparameters and observe their effects without running long training sessions.

3. **Accelerating Training with Mixed Precision**:
   - **Accelerate Library**: Mixed precision training (FP16) is enabled to optimize GPU usage, reduce memory usage, and improve training speed, especially for large models.

4. **Optuna for Hyperparameter Optimization**:
   - **Study Creation**: A study is created using Optuna’s `create_study` function with the objective of minimizing the validation loss.
   - **Optimization Process**: The `study.optimize` method is used to run the trials (in this case, 10 trials), each with a unique set of hyperparameters, to optimize the model.


In [12]:
def objective(trial, model, model_name, train_dataset, eval_dataset):

    # Define hyperparameters to tune
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-4)
    dropout = trial.suggest_uniform('dropout', 0.2, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    weight_decay = trial.suggest_loguniform('weight_decay', 0.01, 0.1)

    # Set dropout in model config (dropout is not set in training_args)
    model.config.attention_probs_dropout_prob = dropout
    model.config.hidden_dropout_prob = dropout

    # Set training_args based on fine-tune hyperparameters
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}_trial_{trial.number}',
        logging_dir=f'./logs/{model_name}_trial_{trial.number}',
        logging_steps=100,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        warmup_steps=300,
        num_train_epochs=5,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        fp16=True,
        seed=42,
    )

    accelerator = Accelerator(mixed_precision="fp16")

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()

    # Evaluation
    eval_result = trainer.evaluate()

    return eval_result['eval_loss']


#### First study:
10 trails base on pre-trained google bert model, best trial: MSE 0.40103545784950256

hyper parameters:
```
learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
batch_size = trial.suggest_categorical('batch_size', [32, 64])
weight_decay = trial.suggest_loguniform('weight_decay', 0.01, 0.1)
n_epoch = 3

```


In [ ]:
study = optuna.create_study(
    study_name="study0205",
    direction='minimize',
    storage='sqlite:////content/optuna_db/study.db',
    load_if_exists=True
)

study.optimize(
    lambda trial: objective(
        trial,
        google_bert_model,
        "google-bert-base_optuna",
        google_bert_train_dataset,
        google_bert_test_dataset
    ),
    n_trials=10
)

[I 2025-02-05 07:44:44,231] A new study created in RDB with name: study0205
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ttzuhsu (ttzuhsu-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

[I 2025-02-05 08:38:51,384] Trial 7 finished with value: 0.4631456732749939 and parameters: {'learning_rate': 0.0002799091545763998, 'dropout': 0.3476373045475534, 'batch_size': 64, 'weight_decay': 0.019870359058480377}. Best is trial 0 with value: 0.40103545784950256.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.097300,0.609962,0.609962,0.376879
2,0.479600,1.001441,1.001441,-0.023046
3,1.092500,0.978949,0.978949,-0.000069


[I 2025-02-05 08:46:05,532] Trial 8 finished with value: 0.6099615097045898 and parameters: {'learning_rate': 0.0006227600324999825, 'dropout': 0.11189933826384837, 'batch_size': 64, 'weight_decay': 0.012148360811198815}. Best is trial 0 with value: 0.40103545784950256.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.179000,1.083870,1.083870,-0.107254
2,0.883000,0.777609,0.777609,0.205615
3,0.585500,0.649330,0.649330,0.336661


[I 2025-02-05 08:53:05,210] Trial 9 finished with value: 0.6493299603462219 and parameters: {'learning_rate': 0.00031823888380612815, 'dropout': 0.1448191125732774, 'batch_size': 32, 'weight_decay': 0.03184021249848639}. Best is trial 0 with value: 0.40103545784950256.


In [ ]:
study_load = optuna.load_study(
    study_name="study0205",
    storage="sqlite:////content/optuna_db/study.db"
)

print(f"Study name: {study.study_name}")
print(f"Study direction: {study.direction}")
print(f"Number of trials: {len(study.trials)}")

Study name: study0205
Study direction: 1
Number of trials: 10


In [ ]:
for trial in study_load.trials:
    print(f"Trial {trial.number}:")
    print(f"  Value: {trial.value}")
    print(f"  Params: {trial.params}")
    print(f"  State: {trial.state}")
    print()

Trial 0:
  Value: 0.40103545784950256
  Params: {'learning_rate': 4.8640483052755485e-05, 'dropout': 0.16892734772481646, 'batch_size': 32, 'weight_decay': 0.03068760271236806}
  State: 1

Trial 1:
  Value: 0.4531651735305786
  Params: {'learning_rate': 0.00014515455816385876, 'dropout': 0.18264516830092262, 'batch_size': 32, 'weight_decay': 0.053120852070534345}
  State: 1

Trial 2:
  Value: 0.42888039350509644
  Params: {'learning_rate': 7.348590114714911e-05, 'dropout': 0.2581720212322267, 'batch_size': 32, 'weight_decay': 0.025390044596121965}
  State: 1

Trial 3:
  Value: 0.4305596649646759
  Params: {'learning_rate': 6.532192802264272e-05, 'dropout': 0.24747800175688678, 'batch_size': 64, 'weight_decay': 0.04558766627001453}
  State: 1

Trial 4:
  Value: 0.4267061650753021
  Params: {'learning_rate': 4.088754286650167e-05, 'dropout': 0.2259343393037393, 'batch_size': 32, 'weight_decay': 0.018632751644231563}
  State: 1

Trial 5:
  Value: 0.4219222664833069
  Params: {'learning_ra

### 2nd trial:
10 trials, hyper parameters:

```
learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
batch_size = trial.suggest_categorical('batch_size', [32, 64])
weight_decay = trial.suggest_loguniform('weight_decay', 0.01, 0.1)
n_epoch = 3

```

best model: trial2 mse: 0.418368935585022

In [11]:
study1 = optuna.create_study(
    study_name="study0206",
    direction='minimize',
    storage='sqlite:////content/optuna_db/study1.db',
    load_if_exists=True
)

study1.optimize(
    lambda trial: objective(
        trial,
        google_bert_model,
        "google-bert-base_optuna",
        google_bert_train_dataset,
        google_bert_test_dataset
    ),
    n_trials=10
)

[I 2025-02-06 06:29:19,554] A new study created in RDB with name: study0206
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ttzuhsu (ttzuhsu-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

[I 2025-02-06 07:15:23,471] Trial 7 finished with value: 0.4154433310031891 and parameters: {'learning_rate': 8.226827197229555e-05, 'dropout': 0.2378410177553461, 'batch_size': 32, 'weight_decay': 0.07306172759079142}. Best is trial 2 with value: 0.405130535364151.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.039500,0.425753,0.425753,0.565062
2,0.046800,0.410553,0.410553,0.580590
3,0.041400,0.419699,0.419699,0.571247


[I 2025-02-06 07:21:25,899] Trial 8 finished with value: 0.4105530381202698 and parameters: {'learning_rate': 1.3107492671970576e-05, 'dropout': 0.3111989221087835, 'batch_size': 64, 'weight_decay': 0.017589304638153815}. Best is trial 2 with value: 0.405130535364151.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.344700,1.023978,1.023978,-0.046069
2,1.074900,0.993177,0.993177,-0.014604
3,1.119500,0.978918,0.978918,-0.000037


[I 2025-02-06 07:26:59,958] Trial 9 finished with value: 0.9789178371429443 and parameters: {'learning_rate': 0.0004332301285870981, 'dropout': 0.39807683431666774, 'batch_size': 32, 'weight_decay': 0.028970557867292298}. Best is trial 2 with value: 0.405130535364151.


In [15]:
for trial in study1.trials:
    print(f"Trial {trial.number}:")
    print(f"  Value: {trial.value}")
    print(f"  Params: {trial.params}")
    print(f"  State: {trial.state}")
    print()

Trial 0:
  Value: 0.406588613986969
  Params: {'learning_rate': 3.303861972058671e-05, 'dropout': 0.478388549159093, 'batch_size': 32, 'weight_decay': 0.03317830696087596}
  State: 1

Trial 1:
  Value: 0.7425116896629333
  Params: {'learning_rate': 0.000595593360102189, 'dropout': 0.34243982912058724, 'batch_size': 64, 'weight_decay': 0.013348753001201176}
  State: 1

Trial 2:
  Value: 0.405130535364151
  Params: {'learning_rate': 6.409999442818139e-05, 'dropout': 0.4581814267188169, 'batch_size': 32, 'weight_decay': 0.011672927754269086}
  State: 1

Trial 3:
  Value: 0.418368935585022
  Params: {'learning_rate': 8.752208511407435e-05, 'dropout': 0.3027074991711967, 'batch_size': 64, 'weight_decay': 0.026416166071885613}
  State: 1

Trial 4:
  Value: 0.40691718459129333
  Params: {'learning_rate': 2.7303462427767877e-05, 'dropout': 0.40884073733232307, 'batch_size': 32, 'weight_decay': 0.05781254828360568}
  State: 1

Trial 5:
  Value: 0.4220510721206665
  Params: {'learning_rate': 0.0

### 3rd trial:
10 trials, hyper parameters:

```
learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-4)
dropout = trial.suggest_uniform('dropout', 0.2, 0.5)
batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
weight_decay = trial.suggest_loguniform('weight_decay', 0.01, 0.1)
n_epoch = 5

```

In [17]:
study2 = optuna.create_study(
    study_name="study0206_2",
    direction='minimize',
    storage='sqlite:////content/optuna_db/study2.db',
    load_if_exists=True
)

study2.optimize(
    lambda trial: objective(
        trial,
        google_bert_model,
        "google-bert-base_optuna2",
        google_bert_train_dataset,
        google_bert_test_dataset
    ),
    n_trials=10
)

[I 2025-02-06 07:36:20,343] Using an existing study with name 'study0206_2' instead of creating a new one.


Epoch,Training Loss,Validation Loss,Mse,R2
1,1.012100,0.983676,0.983676,-0.004898
2,1.012300,0.979204,0.979204,-0.000329
3,1.060300,0.978882,0.978882,-0.000000
4,1.062500,0.979287,0.979287,-0.000414
5,1.004300,0.981483,0.981483,-0.002658


[I 2025-02-06 07:45:10,440] Trial 1 finished with value: 0.9788817167282104 and parameters: {'learning_rate': 5.901410531556497e-05, 'dropout': 0.3589858570912592, 'batch_size': 64, 'weight_decay': 0.015588794923307621}. Best is trial 1 with value: 0.9788817167282104.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.976000,0.984316,0.984316,-0.005551
2,1.048800,0.917858,0.917858,0.062340
3,1.109400,0.910308,0.910308,0.070053
4,0.801500,0.892955,0.892955,0.087780
5,0.887600,0.885105,0.885105,0.095800


[I 2025-02-06 07:56:02,595] Trial 2 finished with value: 0.8851050734519958 and parameters: {'learning_rate': 3.6774344623805236e-06, 'dropout': 0.44690639861682807, 'batch_size': 16, 'weight_decay': 0.03692246250688206}. Best is trial 2 with value: 0.8851050734519958.


...

[I 2025-02-06 08:47:14,672] Trial 8 finished with value: 0.8289816379547119 and parameters: {'learning_rate': 4.935134361808865e-06, 'dropout': 0.21420770123932137, 'batch_size': 64, 'weight_decay': 0.07664155486106117}. Best is trial 6 with value: 0.6846094131469727.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.910200,0.927848,0.927848,0.052134
2,1.118300,0.964122,0.964122,0.015078
3,1.210000,0.982154,0.982154,-0.003343


[I 2025-02-06 08:53:34,521] Trial 9 finished with value: 0.927848219871521 and parameters: {'learning_rate': 3.338569704547321e-05, 'dropout': 0.34814672111293676, 'batch_size': 16, 'weight_decay': 0.08169203923064387}. Best is trial 6 with value: 0.6846094131469727.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.929200,0.845555,0.845555,0.136203
2,0.840700,1.128994,1.128994,-0.153351
3,1.041700,0.977863,0.977863,0.001041


[I 2025-02-06 08:58:49,332] Trial 10 finished with value: 0.8455546498298645 and parameters: {'learning_rate': 8.548489753402403e-05, 'dropout': 0.26739351392114663, 'batch_size': 64, 'weight_decay': 0.027834598316543658}. Best is trial 6 with value: 0.6846094131469727.


In [18]:
study0205 = optuna.load_study(study_name="study0205", storage="sqlite:///optuna_db/study0205.db")

In [ ]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained('./google_finetuned', config = './google_finetuned/config.json')

In [ ]:
study0205.optimize(
    lambda trial: objective(
        trial,
        finetuned_model,
        "google-bert-base_finetuned",
        google_bert_train_dataset,
        google_bert_test_dataset
    ),
  n_startup_trials = 0, n_trials = 5
)